In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib qt5

In [7]:
DIM = 10

In [8]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class OutOfBoundsError(Error):
    """Raised when the input value is too small"""
    def __init__(self, expression, message):
        self.expression = expression
        self.message = message


class InvalidOrientationError(Error):
    """Raised when the the ship orientation is neither 'v' nor 'h'"""
    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

class ShipCollisionError(Error):
    """Raised when the two ships overlap"""
    def __init__(self, expression, message):
        self.expression = expression
        self.message = message



In [9]:
class Player():
    """Class representing a player in a game of battleship."""
    def __init__(self, name="dummy"):
        """a player object is initialized with the name of the player"""
        self.player_name = name
        self.seamap = Map()
        self.enemymap = Map()
        self.fleet = Fleet()
    
    def display_map(self):
        """method to visualize player's map"""
        self.seamap.display()
    
    def place_ships(self):
        """before the game start this method places the ships"""
        for key, ship in self.fleet.ships.items():
            valid_position = False
            xloc = -1
            yloc = -1
            while not valid_position:
                try:
                    print("--------------------------------")
                    xloc = int(input(key + " x coordinate "))
                    yloc = int(input(key + " y coordinate "))
                    orientation = input(key + " orientation ") 
                    position = Position((xloc, yloc), orientation)
                    ship.set_position(position)
                    self.seamap.place_ship(ship)
                    valid_position = True
                except Exception as e: 
                    print(e)
                    print("Invalid Position")
                
    
    def shoot(self):
        """don't forget a docstring"""
        pass
    
    def evaluate(self, other_player_map):
        """don't forget a docstring"""
        pass
    
    def record(self):
        """don't forget a docstring"""
        pass
    
    def has_lost(self):
        return self.fleet.destroyed()

In [20]:
class Map():
    def __init__(self):
        self.karte = np.zeros(shape=(DIM, DIM), dtype=int)
        self.populated_cells = {}
     
    def get_populated_cell_coordinates(self):
        return self.populated_cells.keys()
    
    def place_ship(self, ship):
        x = tmp_x = ship.get_np_x()
        y = tmp_y = ship.get_np_y()
        tmp_list = []
        for i in range(ship.size):
            if ship.get_orientation() == 'v':
                tmp_x = x + i
            else:
                tmp_y = y + i
            tmp_list.append((tmp_x, tmp_y))
        
        if Map.lists_overlap(tmp_list, self.get_populated_cell_coordinates()):
            raise ShipCollisionError("Map", "ship collision error")
        else:
            for tmp in tmp_list:
                self.populated_cells[tmp] = ship.name
                self.karte[tmp[0]][tmp[1]] = 2
            self.display()
            
            
    @classmethod
    def lists_overlap(cls, lst1, lst2):
        return bool([value for value in lst1 if value in lst2])
            
    def display(self):
        #print(self.karte)
        c = mpl.colors.ListedColormap(['lightblue', 'white', 'black', 'red'])
        n = mpl.colors.Normalize(vmin=0,vmax=3)
        ax=np.arange(-0.5, DIM,1)
        plt.matshow(self.karte,cmap=c,norm=n)
        for z in ax:
            plt.plot((z,z),(ax[0],ax[DIM]),'k-')
            plt.plot((ax[0],ax[DIM]),(z,z),'k-')
        plt.axis('off')
        plt.pause(0.1)
        plt.show(block=False)
        #answer = input('Close window to continue... ')
        
        #fig = plt.figure()
        #fig.canvas.mpl_connect('close_event', self.handle_close)
        #plt.show(block=False)
        
    def handle_close(self, evt):
        print('Map Closed')

        

In [21]:
class Coordinates():
    def __init__(self, coordinates):
        x = int(coordinates[0])
        y = int(coordinates[1])
        if x < 1 or x > DIM or y < 1 or x > DIM:
            raise OutOfBoundsError("Coordinates", "value too small or too big")
        else:
            self.np_x = x - 1
            self.np_y = y - 1
        
    def set_np_x(self, x_coordinate):
        self.np_x = x_coordinaterdinate - 1
    
    def set_np_y(self, y_coordinate):
        self.np_y = y_coordinate - 1
    
    def get_np_x(self):
        return self.np_x
    
    def get_np_y(self):
        return self.np_y
    
    def get_x_coordinate(self):
        return (self.np_x + 1)
    
    def get_y_coordinate(self):
        return (self.np_y + 1)
    
    def is_valid(self):
        is_valid = False
        if (0 <= self.np_x <= DIM - 1) and (0 <= self.np_y <= DIM - 1):
            is_valid = True
        return is_valid
    
class Orientation():
    def __init__(self, orientation_str):
        if orientation_str.lower()[0] == 'v':
            self.orientation = 'v'
        elif orientation_str.lower()[0] == 'h':
            self.orientation = 'h'
        else:
            print(orientation_str.lower()[0] )
            raise InvalidOrientationError("Orientation", "invalid orientation")
            
    def get_orientation(self):
        return self.orientation
            
    def is_valid(self):
        return (self.orientation == 'v' or self.orientation == 'h')

class Position():
    def __init__(self, coordinates=None, orientation=None):
        self.coordinates = Coordinates(coordinates)
        self.orientation = Orientation(orientation)
        
    def set_coordinates(self, coordinates):
        self.coordinates = Coordinates(coordinates)
        
    def set_orientation(self, orientation):
        self.orientation = Orientation(orientation)
        
    def is_valid(self):
        return (self.coordinates.is_valid() and self.orientation.is_valid())
    
    def get_np_x(self):
        return self.coordinates.get_np_x()
    
    def get_np_y(self):
        return self.coordinates.get_np_y()

In [22]:
class Ship():
    """Class representing a ship in a game of battleship."""
    def __init__(self, name, size):
        self.size = size
        self.name = name
        self.hits = [0] * self.size
        self.position = None
        
    def set_orientation(self, orientation_str):
        self.position.set_orientation(orientation_str)
        
    def set_coordinates(self, coordinates):
        self.position.set_coordinates(coordinates)
        
    def set_position(self, position):
        self.position = position
        
    def destroyed(self):
        return (self.hits.count(0) == 0)
        
    def get_np_x(self):
        return self.position.get_np_x()
    
    def get_np_y(self):
        return self.position.get_np_y()
    
    def get_orientation(self):
        return self.position.orientation.get_orientation()
    
    def valid_position(self):
        return self.position.is_valid()

In [25]:
class Fleet():
#    ship_properties = [('carrier', 5),\
#                       ('battleship', 4),\
#                       ('cruiser', 3),\
#                       ('submarine', 3),\
#                       ('destroyer', 2)]
    ship_properties = [('carrier', 5), ('battleship', 4), ('cruiser', 3)]
    def __init__(self):
        self.ships = {}
        for properties in Fleet.ship_properties:
            self.ships[properties[0]] = Ship(properties[0], properties[1])
            
            
    def destroyed(self):
        destroyed = True
        for name, ship in self.ships.items():
            if not ship.destroyed():
                destroyed = False
        
        return destroyed
    

In [26]:
%matplotlib qt5
player1 = Player('Alice')
player1.place_ships()

--------------------------------
carrier x coordinate 1
carrier y coordinate 1
carrier orientation h
--------------------------------
battleship x coordinate 3
battleship y coordinate 3
battleship orientation v
--------------------------------
cruiser x coordinate 5
cruiser y coordinate 5
cruiser orientation h


In [ ]:
def main():
    # create players
    player1 = Player('Alice')
    player2 = Player('Bob')
    
    # place ships
    player1.place_ships()
    player2.place_ships()
    
    while (not player1.has_lost()) and (not player2.has_lost()):
        # game loop
        # player 1: shoot
        # player 1: evaluate
        player1.evaluate(player2.seamap)
        # player 1: record
        # player 2: shoot
        # player 2: evaluate
        # player 2: record
        
    if player1.has_lost():
        print("player 2 wins")
    else:
        print("player 1 wins")

In [ ]:
 var = input("name")

In [ ]:
print(var)

In [ ]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt

H = np.array([[1, 2, 3, 4, 17],
          [5, 6, 7, 8, 18],
          [9, 10, 11, 12, 19],
          [13, 14, 15, 16, 20]])

plt.imshow(H)
plt.show()

In [ ]:
karte = np.full((10, 10),0, dtype=int)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors

In [ ]:
colors = ['blue', 'white', 'black', 'red', ]

In [ ]:
#labels = np.random.randint(low=0, high=3, size=20)
#plt.scatter(np.random.rand(20), np.random.rand(20), color=np.array(colors)[labels])
plt.imshow(karte)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
%matplotlib qt

A=[1,1,1,1,1]
B=[0,0,0,0,0]
C=[0,0,3,0,0]
D=[0,0,0,0,0]
E=[2,2,2,2,2]

plateau=np.array([E,D,C,B,A])

c = mpl.colors.ListedColormap(['blue', 'white', 'black', 'red'])
n = mpl.colors.Normalize(vmin=0,vmax=3)

def aff_graph():
    ax=np.arange(-0.5,5,1)
    plt.matshow(plateau,cmap=c,norm=n)
    for z in ax:
        plt.plot((z,z),(ax[0],ax[5]),'k-')
        plt.plot((ax[0],ax[5]),(z,z),'k-')
    plt.axis('off')
    plt.show()
aff_graph()